In [1]:
import numpy
import cv2
import random

In [2]:
from image_preprocessing.scan import *
from image_preprocessing.helper import resize,get_sorted_contours_bounding_box

In [3]:
img = cv2.imread(r'C:\Users\Avinash\Desktop\New folder\OCR\test ocr\hello_world.png')
try:
    img = document_warper(img)
except FourPointException:
    print("ntg")

ntg


In [4]:
print(img.shape)
gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

(577, 700, 3)


In [5]:
edged = edge_detection(img)
cnts,heirarchy = cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)


In [6]:
cnts,bounding_box = get_sorted_contours_bounding_box(cnts)
cv2.imshow('w',gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
chars=[]
boxs=[]
for i,c in enumerate(cnts):
    x,y,w,h = bounding_box[i]
    if (w>=10 and w<=150) and (h>=15 and h<=150):
        roi = gray[y:y+h,x:x+w]
        thresh = cv2.threshold(roi,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        dims = random.randint(45,55)
        thresh = cv2.resize(thresh,(dims,dims),cv2.INTER_CUBIC)
        tH,tW = thresh.shape
        dX = int(max(0,128-tW)/2.0)
        dY = int(max(0,128-tH)/2.0)
        padded = cv2.copyMakeBorder(thresh,top=dY,bottom=dY,right=dX,left=dX,borderType=cv2.BORDER_CONSTANT,value=(255,255,255))
        padded = cv2.resize(padded,(128,128),cv2.INTER_CUBIC)
        padded = padded.astype('float32')/255.
        padded = np.expand_dims(padded,axis=-1)
        boxs.append((x,y,w,h))
        chars.append(padded)

In [8]:
from tensorflow.keras.models import load_model
import json

In [9]:
# thresh=cv2.resize(thresh,(32,32),cv2.INTER_CUBIC)
# cv2.imshow("s",thresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
model = load_model('models/incep nums.h5')

In [10]:
print(type(chars))
chars = np.asarray([c for c in chars],dtype='float32')

for c in chars:
    cv2.imshow('s',c)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

preds = model.predict(chars)

<class 'list'>


In [11]:
with open('class.json','r') as fil:
    classes = json.load(fil)
print(classes)

for p in preds:
    ind = np.argmax(p)
    print(ind,p[ind])

{'0': '30', '1': '31', '2': '32', '3': '33', '4': '34', '5': '35', '6': '36', '7': '37', '8': '38', '9': '39', '10': '41', '11': '42', '12': '43', '13': '44', '14': '45', '15': '46', '16': '47', '17': '48', '18': '49', '19': '4a', '20': '4b', '21': '4c', '22': '4d', '23': '4e', '24': '4f', '25': '50', '26': '51', '27': '52', '28': '53', '29': '54', '30': '55', '31': '56', '32': '57', '33': '58', '34': '59', '35': '5a', '36': '61', '37': '62', '38': '63', '39': '64', '40': '65', '41': '66', '42': '67', '43': '68', '44': '69', '45': '6a', '46': '6b', '47': '6c', '48': '6d', '49': '6e', '50': '6f', '51': '70', '52': '71', '53': '72', '54': '73', '55': '74', '56': '75', '57': '76', '58': '77', '59': '78', '60': '79', '61': '7a'}
4 0.99999976
4 0.9999913
6 0.9999999
6 0.9991875
0 1.0
2 0.7142081
2 0.9999976
0 1.0
6 0.99877316
0 1.0


In [12]:
for (pred,(x,y,w,h)) in zip(preds,boxs):
    idx = int(np.argmax(pred))
    prob = pred[idx]
    idx = str(idx)
    label = chr(int(classes[idx],16))

    print("[INFO] {} -- {:.2f}%".format(label,prob*100))
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,label,(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.2,(0,255,0),2)
cv2.imshow('final',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


[INFO] 4 -- 100.00%
[INFO] 4 -- 100.00%
[INFO] 6 -- 100.00%
[INFO] 6 -- 99.92%
[INFO] 0 -- 100.00%
[INFO] 2 -- 71.42%
[INFO] 2 -- 100.00%
[INFO] 0 -- 100.00%
[INFO] 6 -- 99.88%
[INFO] 0 -- 100.00%


In [13]:
cv2.imwrite(r"C:\Users\Avinash\Desktop\New folder\OCR\shit test nums.jpg",img)

True